In [224]:
import pandas as pd

df = pd.read_xml('TEST 22010HB.xml', encoding='utf-16')

In [225]:
df = df.sort_values(by='Name', key=lambda col: col.str.lower()).reset_index(drop=True)

In [226]:
df[['X', 'Y', 'Z']] = df['Coordinate'].str.split('  ', expand = True)

In [227]:
df.drop('Coordinate', axis = 1, inplace = True)

In [228]:

for col in ['X', 'Y', 'Z']:
    df[col] = df[col].apply(lambda x : float(x))

In [229]:
df.rename(columns={'Name':'Repère'}, inplace = True)

In [230]:
values = df["Repère"].values.tolist()
for value in values:
    occ = [x for x in values if x[2:] == value[2:]]

    if len(occ) != 2:
        print(value)

In [231]:
df.set_index('Repère', inplace = True)


In [232]:
df_dif = pd.DataFrame(columns=['Repère', 'X\n+ : Décalage à gauche\n- : Décalage à droite', 'Y\n+ : recule\n- : avancée', 'Z\n+ : impaction\n- : épaction'])
df_dif

break_point_inf = [value for value in df.index.tolist() if value[:2] == '00']
break_point_sup = [value for value in df.index.tolist() if value[:2] != '00']

for value in df.index[:len(break_point_inf)]:
    row = [value[2:]]
    row.append(float(df[df.index == (value[:1] + '1' + value[2:])]['X'].values - df[df.index == value]['X'].values))
    row.append(float(df[df.index == (value[:1] + '1' + value[2:])]['Y'].values - df[df.index == value]['Y'].values))
    row.append(float(df[df.index == (value[:1] + '1' + value[2:])]['Z'].values - df[df.index == value]['Z'].values))
    df_dif.loc[len(df_dif)] = row

df_dif.set_index('Repère', inplace = True)
    

In [233]:
df_before = df.loc[:break_point_inf[-1]].reset_index()
df_after = df.loc[break_point_sup[0]:].reset_index()
df_before['Repère'] = df_before['Repère'].apply(lambda x : x[2:])
df_after['Repère'] = df_after['Repère'].apply(lambda x : x[2:])
df_before.set_index('Repère', inplace=True)
df_after.set_index('Repère', inplace=True)

In [235]:

with pd.ExcelWriter('test.xlsx', engine='xlsxwriter') as writer:

    df_before.to_excel(writer, sheet_name='Déplacement', startrow=1 , startcol=0) 

    df_after.to_excel(writer, sheet_name='Déplacement', startrow=1 , startcol=df.shape[1]+3)

    df_dif.style.to_excel(writer, sheet_name='Déplacement', startrow=1, startcol=df.shape[1]*2+6) 

    format = writer.book.add_format({'num_format': '0.0'})
    format_neg = writer.book.add_format({'num_format': '0.0', 'color' : 'red'})
    format_pos = writer.book.add_format({'num_format': '0.0', 'color' : 'green'})

    merge_format = writer.book.add_format({"bold": 1, "border": 1, "align": "center", "valign": "vcenter", "fg_color": "#008db9"})

    writer.sheets['Déplacement'].conditional_format(f'{chr(65 + df.shape[1]*2+7)}2:{chr(65 + df.shape[1]*2+6 + df_dif.shape[1])}{df_dif.shape[0]+2}', {'type' : 'cell', 'criteria' : '<', 'value' : 0, 'format' : format_neg})
    writer.sheets['Déplacement'].conditional_format(f'{chr(65 + df.shape[1]*2+7)}2:{chr(65 + df.shape[1]*2+6 + df_dif.shape[1])}{df_dif.shape[0]+2}', {'type' : 'cell', 'criteria' : '>=', 'value' : 0, 'format' : format_pos})

    writer.sheets['Déplacement'].add_table(f'A2:{chr(65 + df.shape[1])}{len(break_point_inf)+2}', {'style': 'Table Style Medium 2','header_row': False})
    writer.sheets['Déplacement'].add_table(f'{chr(65 + df.shape[1]+3)}2:{chr(65 + df.shape[1]*2+3)}{len(break_point_sup)+2}', {'style': 'Table Style Medium 2','header_row': False})
    writer.sheets['Déplacement'].add_table(f'{chr(65 + df.shape[1]*2+6)}2:{chr(65 + df.shape[1]*2+6 + df_dif.shape[1])}{df_dif.shape[0]+2}', {'style': 'Table Style Light 9','header_row': False})

    writer.sheets['Déplacement'].merge_range(f"A1:{chr(65 + df.shape[1])}1", "Avant planification", merge_format)
    writer.sheets['Déplacement'].merge_range(f"{chr(65 + df.shape[1]+3)}1:{chr(65 + df.shape[1]*2+3)}1", "Après planification", merge_format)
    writer.sheets['Déplacement'].merge_range(f"{chr(65 + df.shape[1]*2+6)}1:{chr(65 + df.shape[1]*3+6)}1", "Déplacement prévu", merge_format)

    writer.sheets['Déplacement'].set_column(f'B:{chr(65 + df.shape[1])}', None, format)
    writer.sheets['Déplacement'].set_column(f'{chr(65 + df.shape[1]+3)}:{chr(65 + df.shape[1]+3 + df_dif.shape[1])}', None, format)

    writer.sheets['Déplacement'].set_row(1, 40)

    writer.sheets['Déplacement'].autofit()
